In [1]:
# Agregamos la ruta donde se encuentran las funciones de utilidad
import sys
sys.path.insert(0, '../utils/')

# Import de librerías necesarias
import cleaning_functions as fnc
from variables import *

import pandas as pd
import numpy as np

In [2]:
# Leemos el fichero generado en el paso anterior para realizar la limpieza de datos
df = pd.read_csv(TURISMO_ECO_ZERO_FILENAME_RAW, sep = CSV_SEPARADOR_COLUMNAS)

C:\Users\justo\AppData\Local\Temp\ipykernel_8028\1479321217.py:2: DtypeWarning: Columns (15,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(TURISMO_ECO_ZERO_FILENAME_RAW, sep = CSV_SEPARADOR_COLUMNAS)


In [3]:
# Cambiamos el nombre de la columna 'Unnamed: 0' generada con el ID del archivo anterior y lo establecemos como indice.
df.rename(columns = {"Unnamed: 0":"ID"}, inplace = True)
df.set_index('ID',inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1620165 entries, 13 to 37890848
Data columns (total 22 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   PROVINCIA        1620165 non-null  int64  
 1   MARCA            1620165 non-null  object 
 2   MODELO           1620165 non-null  object 
 3   PROVINCIA_MATR   1620146 non-null  float64
 4   FECHA_MATR       1620165 non-null  object 
 5   FECHA_PRIM_MATR  1620165 non-null  object 
 6   TIPO_DGT         1620165 non-null  object 
 7   CAT_EURO         1620029 non-null  object 
 8   RENTING          1611275 non-null  object 
 9   TARA             1620165 non-null  int64  
 10  PESO_MAX         1620165 non-null  int64  
 11  CILINDRADA       1620165 non-null  int64  
 12  POTENCIA         1620165 non-null  object 
 13  KW               1620160 non-null  object 
 14  PROPULSION       1619778 non-null  object 
 15  CATELECT         1476571 non-null  object 
 16  CONSUMO          1620

In [4]:
df.head()

,PROVINCIA,MARCA,MODELO,PROVINCIA_MATR,FECHA_MATR,FECHA_PRIM_MATR,TIPO_DGT,CAT_EURO,RENTING,TARA,...,POTENCIA,KW,PROPULSION,CATELECT,CONSUMO,AUTONOMIA,ALIMENTACION,TIPO_DISTINTIVO,EMISIONES_EURO,EMISIONES_CO2
ID,,,,,,,,,,,,,,,,,,,,,
13,21,RENAULT,ARKANA E-TECH,40.0,28/06/2022,28/06/2022,TURISMOS,M1,N,1435,...,"11,64","69,00",0.0,HEV,0.0,0.0,M,ECO,EURO 6AP,108.0
22,17,HYUNDAI,KONA,18.0,07/03/2022,07/03/2022,TURISMOS,M1,N,1376,...,"11,56","77,20",0.0,HEV,0.0,0.0,M,ECO,EURO 6AP,112.0
36,3,KIA,NIRO,1.0,28/03/2018,28/03/2018,TURISMOS,M1,N,1425,...,"11,56","77,20",0.0,HEV,0.0,0.0,M,ECO,EURO 6W,88.0
106,13,LEXUS,LEXUS NX300H,28.0,29/03/2016,29/03/2016,TURISMOS,M1,N,1815,...,"0,15","114,00",0.0,HEV,0.0,0.0,M,ECO,EURO 6W,121.0
143,3,FORD,PUMA,15.0,16/12/2021,16/12/2021,TURISMOS,M1,N,1238,...,"7,82","91,90",0.0,HEV,0.0,0.0,M,ECO,EURO 6AP,129.0


In [5]:
# Comprobamos las columnas con valores nulos
print(df.columns[df.isna().any()].tolist())
print(df.isnull().sum())

# ['PROVINCIA_MATR', 'CAT_EURO', 'RENTING', 'KW', 'PROPULSION', 'CATELECT', 'CONSUMO', 'AUTONOMIA', 'ALIMENTACION', 'EMISIONES_EURO', 'EMISIONES_CO2']

['PROVINCIA_MATR', 'CAT_EURO', 'RENTING', 'KW', 'PROPULSION', 'CATELECT', 'CONSUMO', 'AUTONOMIA', 'ALIMENTACION', 'EMISIONES_EURO', 'EMISIONES_CO2']
PROVINCIA               0
MARCA                   0
MODELO                  0
PROVINCIA_MATR         19
FECHA_MATR              0
FECHA_PRIM_MATR         0
TIPO_DGT                0
CAT_EURO              136
RENTING              8890
TARA                    0
PESO_MAX                0
CILINDRADA              0
POTENCIA                0
KW                      5
PROPULSION            387
CATELECT           143594
CONSUMO                 5
AUTONOMIA           10607
ALIMENTACION        42675
TIPO_DISTINTIVO         0
EMISIONES_EURO       4153
EMISIONES_CO2       18263
dtype: int64


### Campos: FECHA_MATR, FECHA_PRIM_MATR

Pasamos los campos de fecha, que se han cargado como 'object' a datetime (FECHA_MATR, FECHA_PRIM_MATR)

In [6]:
# Patrón para transformar a fechas los campos FECHA_MATR, FECHA_PRIM_MATR
patron = "%d/%m/%Y"

In [7]:
# ******* Campo: FECHA_MATR *******
df.FECHA_MATR = pd.to_datetime(df.FECHA_MATR, format = patron)

# ******* Campo: FECHA_PRIM_MATR *******
df.FECHA_PRIM_MATR = pd.to_datetime(df.FECHA_PRIM_MATR, format = patron)

### Campos: MARCA Y MODELO 

Se ha identificado que para los campos MARCA y MODELO muchos valores se especifica carácter "¡". Vamos a sustituir este valor por X para ponerlo como desconocido

In [8]:
len(df.loc[(df.MARCA == "¡") | (df.MODELO == "¡")])

244375

In [9]:
df.loc[df.MARCA == "¡", "MARCA"] = VALUE_SIN_DETERMINAR
df.loc[df.MODELO == "¡", "MODELO"] = VALUE_SIN_DETERMINAR

In [10]:
display(df.loc[(df.MARCA == VALUE_SIN_DETERMINAR) | (df.MODELO == VALUE_SIN_DETERMINAR)])

,PROVINCIA,MARCA,MODELO,PROVINCIA_MATR,FECHA_MATR,FECHA_PRIM_MATR,TIPO_DGT,CAT_EURO,RENTING,TARA,...,POTENCIA,KW,PROPULSION,CATELECT,CONSUMO,AUTONOMIA,ALIMENTACION,TIPO_DISTINTIVO,EMISIONES_EURO,EMISIONES_CO2
ID,,,,,,,,,,,,,,,,,,,,,
621,20,MERCEDES-BENZ,* ND *,28.0,2022-07-13,2022-07-13,TURISMOS,M1,N,1725,...,"11,19","150,00",0.0,HEV,0.0,0.0,M,ECO,EURO 6AP,148.0
792,8,HONDA,* ND *,5.0,2020-10-28,2020-10-28,TURISMOS,M1,N,1610,...,"0,13","107,00",0.0,HEV,0.0,0.0,M,ECO,EURO 6DG,120.0
996,3,SEAT,* ND *,1.0,2021-08-31,2021-08-31,TURISMOS,M1,N,0,...,"10,72","110,00",0.0,PHEV,148.0,6300.0,M,CERO,EURO 6AP,26.0
1194,8,MG,* ND *,5.0,2023-02-07,2023-02-07,TURISMOS,M1,N,1775,...,"11,16","119,00",0.0,PHEV,240.0,5200.0,M,CERO,EURO 6AP,43.0
1245,12,LEXUS,* ND *,15.0,2017-10-04,2017-10-04,TURISMOS,M1,N,1735,...,"15,20","133,00",0.0,HEV,0.0,0.0,M,ECO,EURO 6W,113.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37889901,28,CITROEN,* ND *,28.0,2023-11-22,2023-11-22,TURISMOS,M1,N,1770,...,"11,64","133,00",0.0,PHEV,166.0,6500.0,M,CERO,EURO 6AP,29.0
37889912,28,HONDA,* ND *,28.0,2011-04-18,2011-04-18,TURISMOS,M1,N,1195,...,"11,19","84,00",0.0,HEV,0.0,NaN,NaN,ECO,EURO 5F,117.0
37890201,28,CITROEN,* ND *,47.0,2017-07-27,2017-07-27,TURISMOS,M1,N,1164,...,"11,59","85,00",6.0,NaN,0.0,0.0,M,ECO,EURO 6,140.0


### Campos: PROVINCIA, PROVINCIA_MATR

Pasamos los campos de provincias, que tienen identificadores a texto (PROVINCIA, PROVINCIA_MATR)


In [11]:
dict_provincias = { '1':'Álava','2':'Albacete', '3':'Alicante', '4':'Almería', '5':'Ávila', '6':'Badajoz', '7':'Islas Balears', '8':'Barcelona', '9':'Burgos', 
                   '10':'Cáceres', '11':'Cádiz', '12':'Castellón', '13':'Ciudad Real', '14':'Córdoba', '15':'La Coruña', '16':'Cuenca', '17':'Girona', '18':'Granada', '19':'Guadalajara',
                   '20':'Gipuzkoa', '21':'Huelva', '22':'Huesca', '23':'Jaén', '24':'León', '25':'Lleida', '26':'La Rioja', '27':'Lugo', '28':'Madrid', '29':'Málaga',
                   '30':'Murcia', '31':'Navarra', '32':'Ourense', '33':'Asturias', '34':'Palencia', '35':'Las Palmas', '36':'Pontevedra', '37':'Salamanca', '38':'Santa Cruz de Tenerife', '39':'Cantabria',
                   '40':'Segovia', '41':'Sevilla', '42':'Soria', '43':'Tarragona', '44':'Teruel', '45':'Toledo', '46':'Valencia', '47':'Valladolid', '48':'Vizcaya', '49':'Zamora', 
                   '50':'Zaragoza', '51':'Ceuta', '52':'Melilla', '99':'Extranjero', '100':'* Desconocido *'
                   }


In [12]:
# ******* Campo: PROVINCIA *******

display(df.PROVINCIA.value_counts(dropna=False))

# No hay valores nulos, hacemos la correspondencia directamente con el diccionario de provincias
fnc.replace_values_from_master(df, "PROVINCIA", dict_provincias)

display(df.PROVINCIA.value_counts(dropna=False))

PROVINCIA
28    609613
8     194912
46     71527
3      54732
29     45141
41     39736
7      35788
35     34774
30     33751
48     26473
15     25835
17     25739
11     24701
43     24668
33     23264
50     22274
38     20411
45     20032
18     18014
36     17710
12     15988
20     15218
31     14936
4      14506
39     14488
47     14074
14     11634
25     11557
6      10947
24     10078
1       9075
21      8709
23      8393
19      8162
9       7978
13      7857
27      7319
2       7293
26      7279
37      6309
10      6043
32      5590
22      4683
40      3506
16      3160
5       3156
34      3074
49      2901
44      2265
52      1860
42      1711
51      1321
Name: count, dtype: int64

PROVINCIA
Madrid                    609613
Barcelona                 194912
Valencia                   71527
Alicante                   54732
Málaga                     45141
Sevilla                    39736
Islas Balears              35788
Las Palmas                 34774
Murcia                     33751
Vizcaya                    26473
La Coruña                  25835
Girona                     25739
Cádiz                      24701
Tarragona                  24668
Asturias                   23264
Zaragoza                   22274
Santa Cruz de Tenerife     20411
Toledo                     20032
Granada                    18014
Pontevedra                 17710
Castellón                  15988
Gipuzkoa                   15218
Navarra                    14936
Almería                    14506
Cantabria                  14488
Valladolid                 14074
Córdoba                    11634
Lleida                     11557
Badajoz                    10947
León                       10078


In [13]:
# ******* Campo: PROVINCIA_MATR *******

display(df.PROVINCIA_MATR.value_counts(dropna=False))

#Rellenamos los nulos con el valor "100" (Desconocido) para los 19 valores nulos
# Hacemos la correspondencia directamente con el diccionario de provincias
fnc.fillna_and_replace_values_from_master(df, "PROVINCIA_MATR", dict_provincias, 100)

display(df.PROVINCIA_MATR.value_counts(dropna=False))

PROVINCIA_MATR
28.0    664270
5.0     207624
1.0      62828
48.0     47414
40.0     38590
29.0     37668
17.0     34234
36.0     29968
9.0      27023
7.0      25674
31.0     25574
47.0     24545
51.0     24081
39.0     22482
10.0     21599
13.0     21173
18.0     20737
44.0     19528
33.0     19472
38.0     17845
46.0     16531
43.0     13940
15.0     12982
19.0     12811
32.0     12176
49.0     12169
6.0      10783
37.0     10657
8.0      10421
24.0      9807
3.0       9411
23.0      9340
25.0      9303
50.0      8763
27.0      7392
2.0       7248
26.0      5924
21.0      5637
14.0      5563
34.0      5561
22.0      4798
11.0      4529
20.0      4474
41.0      3016
52.0      2752
4.0       2430
45.0      2281
30.0      1801
42.0      1532
35.0      1439
12.0      1273
16.0      1073
NaN         19
Name: count, dtype: int64

PROVINCIA_MATR
Madrid                    664270
Ávila                     207624
Álava                      62828
Vizcaya                    47414
Segovia                    38590
Málaga                     37668
Girona                     34234
Pontevedra                 29968
Burgos                     27023
Islas Balears              25674
Navarra                    25574
Valladolid                 24545
Ceuta                      24081
Cantabria                  22482
Cáceres                    21599
Ciudad Real                21173
Granada                    20737
Teruel                     19528
Asturias                   19472
Santa Cruz de Tenerife     17845
Valencia                   16531
Tarragona                  13940
La Coruña                  12982
Guadalajara                12811
Ourense                    12176
Zamora                     12169
Badajoz                    10783
Salamanca                  10657
Barcelona                  10421
León                        

### TIPO_DGT

Para este campo no se han identificado medidas de limpieza a realizar

In [14]:
# ******* Campo: TIPO_DGT *******

# No hay que hacer nada

### CAT_EURO

Los valores nulos se sutsituyen por la MODA de los registros con igual MARCA, MODELO, PROPULSION y CATELECT

In [15]:
# ******* Campo: CAT_EURO *******

display(df.CAT_EURO.value_counts(dropna=False))
1
fnc.replace_null_from_master_columns(df, "CAT_EURO", ["MARCA","MODELO","PROPULSION","CATELECT"], VALUE_SIN_DETERMINAR)

display(df.CAT_EURO.value_counts(dropna=False))

CAT_EURO
M1     1619771
M1G        159
NaN        136
N1          89
ATV          3
L7E          2
*55          2
*05          1
N1G          1
OTR          1
Name: count, dtype: int64

Estableciendo mmoda del campo 'CAT_EURO' para las columnas ['MARCA', 'MODELO', 'PROPULSION', 'CATELECT']


CAT_EURO
M1        1619793
M1G           160
* ND *        113
N1             89
ATV             3
L7E             2
*55             2
*05             1
N1G             1
OTR             1
Name: count, dtype: int64

### RENTING

Tiene 8890 valores nulos. Esbblecemos el valor 'x' a a todos los nulos.


In [16]:
# ******* Campo: RENTING *******

display(df.RENTING.value_counts(dropna=False))

# Ya que hay valores nulos, establecemos 'X' como valor desconocido
fnc.fillna(df, "RENTING", VALUE_SIN_DETERMINAR)

display(df.RENTING.value_counts(dropna=False))

RENTING
N      1369803
S       241472
NaN       8890
Name: count, dtype: int64

RENTING
N         1369803
S          241472
* ND *       8890
Name: count, dtype: int64

### TARA

Para este campo no se han identificado medidas de limpieza a realizar

In [17]:
# ******* Campo: TARA *******

# No hay que hacer nada

### PESO_MAX

Para este campo no se han identificado medidas de limpieza a realizar

In [18]:
# ******* Campo: PESO_MAX *******

# No hay que hacer nada

### CILINDRADA

Para este campo no se han identificado medidas de limpieza a realizar

In [19]:
# ******* Campo: CILINDRADA *******

# No hay que hacer nada

### POTENCIA

Se han encontrado valores con ",". Las reemplazamos por "." para que se considere una columna numérica

In [20]:
# ******* Campo: POTENCIA *******

# A traves de la siguiente consulta vamos identificando los valores no válidos del campo
#df.POTENCIA.apply(pd.to_numeric, errors="raise")

# Se han encontrado algunos valores con "," para el campo autonomía. Los sustituimos por "."
df["POTENCIA"] = df.POTENCIA.str.replace(",",".")

#Convertimos la columna a float
df["POTENCIA"] = df.POTENCIA.apply(pd.to_numeric, downcast="float", errors="raise")

### KW Y CONSUMO

Se han encontrado 5 registros nulos para las columnas KW y CONSUMO. 
Tiene 8890 valores nulos. Esbblecemos el valor 'x' a a todos los nulos.

In [21]:
# ******* Campo: KW Y CONSUMO *******
df.loc[(df.KW.isna()) | (df.CONSUMO.isna())]

,PROVINCIA,MARCA,MODELO,PROVINCIA_MATR,FECHA_MATR,FECHA_PRIM_MATR,TIPO_DGT,CAT_EURO,RENTING,TARA,...,POTENCIA,KW,PROPULSION,CATELECT,CONSUMO,AUTONOMIA,ALIMENTACION,TIPO_DISTINTIVO,EMISIONES_EURO,EMISIONES_CO2
ID,,,,,,,,,,,,,,,,,,,,,
6968586,Islas Balears,BMW,* ND *,* Desconocido *,1980-05-15,1980-05-15,TURISMOS,* ND *,* ND *,1530,...,0.22,NaN,2.0,NaN,NaN,NaN,NaN,CERO,NaN,NaN
11011157,Barcelona,SEAT,131 1600,* Desconocido *,1977-01-26,1977-01-26,TURISMOS,* ND *,* ND *,1000,...,0.12,NaN,2.0,NaN,NaN,NaN,NaN,CERO,NaN,NaN
15237013,Barcelona,SEAT,* ND *,* Desconocido *,1966-12-01,1966-12-01,TURISMOS,* ND *,* ND *,1190,...,11.00,NaN,2.0,NaN,NaN,NaN,NaN,CERO,NaN,NaN
15935206,Huesca,CHRYSLER,* ND *,* Desconocido *,1975-04-10,1975-04-10,TURISMOS,* ND *,* ND *,1100,...,0.13,NaN,2.0,NaN,NaN,NaN,NaN,CERO,NaN,NaN
20305653,Madrid,SIN MARCA,* ND *,* Desconocido *,1989-04-19,1989-04-19,TURISMOS,* ND *,* ND *,1140,...,5.00,NaN,2.0,NaN,NaN,NaN,NaN,CERO,NaN,NaN


In [22]:
# Ya que coinciden los 5 registros nulos para ambas columnas, decicimos eliminarlos
df.drop(df.loc[(df.KW.isna()) | (df.CONSUMO.isna())].index, inplace = True)

### KW

Se han encontrado valores con ",". Las reemplazamos por "." para que se considere una columna numérica

In [23]:
# ******* Campo: KW *******

# A traves de la siguiente consulta vamos identificando los valores no válidos del campo
#df.KW.apply(pd.to_numeric, errors="raise")

# Se han encontrado algunos valores con "," para el campo autonomía. Los sustituimos por "."
df["KW"] = df.KW.str.replace(",",".")

#Convertimos la columna a float
df["KW"] = df.KW.apply(pd.to_numeric, downcast="float", errors="raise")

### PROPULSION

Tiene 387 valores nulos. Creamos una nueva categoría 'X' como 'Desconocido' y asignamos el valor a todos los nulos.
Posteriormente asignamos el la categoría correspondiente en función del identificador a partir del diccionario 'dict_propulsion'

In [24]:
dict_propulsion = {'0':'Gasolina', '1':'Diésel', '2':'Eléctrico', '3':'Otros', '4':'Butano', '5':'Solar', '6':'Gas Licuado de Petróleo', '7':'Gas Natural Comprimido', 
                   '8':'Gas Natural Licuado', '9':'Hidrógeno', 'A':'Biometano', 'B':'Etanol', 'C':'Biodiesel', 'D':'Diésel', 'G':'Gasolina', VALUE_SIN_DETERMINAR: VALUE_SIN_DETERMINAR
}


In [25]:
# ******* Campo: PROPULSION *******

display(df.PROPULSION.value_counts(dropna=False))

# Ya que hay valores nulos, creamos una nueva categoría X de variables.VALUE_SIN_DETERMINAR. Establecemos los nulos a esta categoría
# Actualizamos los valores según el diccionario 'dict_propulsion' a partir del identificador
fnc.fillna_and_replace_values_from_master(df, "PROPULSION", dict_propulsion, VALUE_SIN_DETERMINAR)

display(df.PROPULSION.value_counts(dropna=False))

PROPULSION
0.0    861461
0.0    278489
2.0    117637
6.0     86305
1.0     83460
0       61844
2.0     38086
6.0     27972
1.0     26826
7.0     12086
2        8360
6        6240
1        5887
7.0      3974
7         862
NaN       387
9.0        79
8.0        72
4.0        37
9.0        25
8.0        22
4.0        12
9          10
C           7
A           6
4           5
8           5
G           3
E           1
Name: count, dtype: int64

PROPULSION
Gasolina                   1201797
Eléctrico                   164083
Gas Licuado de Petróleo     120517
Diésel                      116173
Gas Natural Comprimido       16922
* ND *                         388
Hidrógeno                      114
Gas Natural Licuado             99
Butano                          54
Biodiesel                        7
Biometano                        6
Name: count, dtype: int64

### CATELECT

Tiene 143589 valores nulos. Establecemos el valor 'X' como 'Desconocido' y asignamos el valor a todos los nulos.
Posteriormente asignamos el la categoría correspondiente en función del identificador a partir del diccionario 'dict_catelect'

In [26]:
dict_catelect = {'REEV':'(REEV) Eléctrico de Autonomía Extendida', 'HEV':'(HEV) Eléctrico Híbrido', 'BEV':'(BEV) Eléctrico de Batería', 
                 'PHEV':'(PHEV) Eléctrico Híbrido Enchufable', 'FCEV':'(FCEV) Pila de combustible-hidrógeno', VALUE_SIN_DETERMINAR: VALUE_SIN_DETERMINAR
}

In [27]:
# ******* Campo: CATELECT *******

display(df.CATELECT.value_counts(dropna=False))

# Ya que hay valores nulos, establecemos 'X' como 'Desconocido'. Establecemos los nulos a esta categoría
# Actualizamos los valores según el diccionario 'dict_propulsion' a partir del identificador
fnc.fillna_and_replace_values_from_master(df, "CATELECT", dict_catelect, VALUE_SIN_DETERMINAR)

display(df.CATELECT.value_counts(dropna=False))

CATELECT
HEV     1162352
PHEV     163783
BEV      148487
NaN      143589
REEV       1916
FCEV         33
Name: count, dtype: int64

CATELECT
(HEV) Eléctrico Híbrido                    1162352
(PHEV) Eléctrico Híbrido Enchufable         163783
(BEV) Eléctrico de Batería                  148487
* ND *                                      143589
(REEV) Eléctrico de Autonomía Extendida       1916
(FCEV) Pila de combustible-hidrógeno            33
Name: count, dtype: int64

### CONSUMO

Tras haber eliminado los registros con valor nulo relacionados con KW, Para este campo no se han identificado medidas de limpieza a realizar

In [28]:
# ******* Campo: CONSUMO *******

# No hay que hacer nada

### AUTONOMIA

Completamos los valores nulos con la moda correspondiente a las columnas ["MARCA","MODELO","PROPULSION","CATELECT"] y si no se encuentra valor, establecemos el valor a NUM_VALUE_SIN_DETERMINAR para poder idenfificar en el análisis posterior los valores que eran nulos

In [29]:
# ******* Campo: EMISIONES_EURO *******

# A traves de la siguiente consulta vamos identificando los valores no válidos del campo
#df.AUTONOMIA.apply(pd.to_numeric, errors="raise")

# Se han encontrado algunos valores con "," para el campo autonomía. Los sustituimos por "."
df["AUTONOMIA"] = df.AUTONOMIA.str.replace(",",".")

# También se han encontrado valores que empiezan por "0 ". Suponemos que es error y elimimnamos esa cadena
df["AUTONOMIA"] = df.AUTONOMIA.str.replace("0 ","")

#Convertimos la columna a float
df["AUTONOMIA"] = df.AUTONOMIA.apply(pd.to_numeric, downcast="float", errors="raise")

In [30]:
# ******* Campo: AUTONOMIA *******

display(df.AUTONOMIA.value_counts(dropna=False))

fnc.replace_null_from_master_columns(df, "AUTONOMIA", ["MARCA","MODELO","PROPULSION","CATELECT"], NUM_VALUE_SIN_DETERMINAR)

display(df.AUTONOMIA.value_counts(dropna=False))

AUTONOMIA
NaN         1197022
0.0          340222
6200.0         3807
5200.0         2772
6100.0         2726
             ...   
58800.0           1
71000.0           1
660000.0          1
63900.0           1
1000.0            1
Name: count, Length: 643, dtype: int64

Estableciendo mmoda del campo 'AUTONOMIA' para las columnas ['MARCA', 'MODELO', 'PROPULSION', 'CATELECT']


AUTONOMIA
 0.000000        1100607
-10.000000        278341
 6200.000000       12863
 60200.000000      10338
 6100.000000        9960
                  ...   
 24900.000000          1
 41.799999             1
 65.000000             1
 70000.000000          1
 1000.000000           1
Name: count, Length: 643, dtype: int64

### ALIMENTACION

Tiene 42675 valores nulos. Esbblecemos el valor '0' como 'Sin informar' y asignamos el valor a todos los nulos.
Posteriormente asignamos el la categoría correspondiente en función del identificador a partir del diccionario 'dict_alimentacion'

In [31]:
dict_alimentacion = {'0':'Sin informar', 'B':'Bifuel', 'F':'Flexifuel', 'M':'Monofuel' }


In [32]:
# ******* Campo: ALIMENTACION *******

display(df.ALIMENTACION.value_counts(dropna=False))

# Ya que hay valores nulos, establecemos '0' como 'Sin informar'. Establecemos los nulos a esta categoría
# Actualizamos los valores según el diccionario 'dict_propulsion' a partir del identificador
fnc.fillna_and_replace_values_from_master(df, "ALIMENTACION", dict_alimentacion, 0)

display(df.ALIMENTACION.value_counts(dropna=False))

ALIMENTACION
M      1440070
B       137391
NaN      42670
F           29
Name: count, dtype: int64

ALIMENTACION
Monofuel        1440070
Bifuel           137391
Sin informar      42670
Flexifuel            29
Name: count, dtype: int64

### TIPO_DISTINTIVO

Para este campo no se han identificado medidas de limpieza a realizar

In [33]:
# ******* Campo: TIPO DISTINTIVO *******

# No hay que hacer nada

### EMISIONES_EURO

Los valores nulos se sutsituyen por la MODA de los registros con igual MARCA, MODELO, PROPULSION y CATELECT

In [34]:
# ******* Campo: EMISIONES_EURO *******

display(df.EMISIONES_EURO.value_counts(dropna=False))

fnc.replace_null_from_master_columns(df, "EMISIONES_EURO", ["MARCA","MODELO","PROPULSION","CATELECT"], VALUE_SIN_DETERMINAR)

display(df.EMISIONES_EURO.value_counts(dropna=False))

EMISIONES_EURO
EURO 6AP    924586
EURO 6W     172683
EURO 6DG    125601
EURO AX      96450
EURO 6AG     55101
             ...  
EURO6X           1
EURO 6M          1
EURO6-AA         1
FASE V           1
EURO6AP          1
Name: count, Length: 80, dtype: int64

Estableciendo mmoda del campo 'EMISIONES_EURO' para las columnas ['MARCA', 'MODELO', 'PROPULSION', 'CATELECT']


EMISIONES_EURO
EURO 6AP    924586
EURO 6W     172686
EURO 6DG    125601
EURO AX      96770
EURO 6AG     55101
             ...  
EURO6X           1
EURO 6M          1
EURO6-AA         1
FASE V           1
EURO6AP          1
Name: count, Length: 80, dtype: int64

### EMISIONES C02

Para los nulos, establecemos el valor de la moda según MARCA, MODELO y PROPULSION o el valor NUM_VALUE_SIN_DETERMINAR para identificar los valores que eran nulos

In [35]:
# ******* Campo: EMISIONES_CO2 *******

display(df.EMISIONES_CO2.value_counts(dropna=False))

fnc.replace_null_from_master_columns(df, "EMISIONES_CO2", ["MARCA","MODELO","PROPULSION","CATELECT"], NUM_VALUE_SIN_DETERMINAR)

display(df.EMISIONES_CO2.value_counts(dropna=False))


EMISIONES_CO2
0.00      153657
101.00     40892
86.00      38414
119.00     36756
120.00     36689
           ...  
134.40         1
272.00         1
337.00         1
152.23         1
327.00         1
Name: count, Length: 383, dtype: int64

Estableciendo mmoda del campo 'EMISIONES_CO2' para las columnas ['MARCA', 'MODELO', 'PROPULSION', 'CATELECT']


EMISIONES_CO2
0.00      154761
101.00     40892
86.00      38414
119.00     36756
120.00     36689
           ...  
134.40         1
272.00         1
337.00         1
152.23         1
327.00         1
Name: count, Length: 383, dtype: int64

# RESUMEN FINAL

In [36]:
# Guardamos el resultado de toda la eliminación de nulos y estandarización de valores en un nuevo archivo
df.to_csv(TURISMO_ECO_ZERO_FILENAME_FINAL, sep=CSV_SEPARADOR_COLUMNAS)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1620160 entries, 13 to 37890848
Data columns (total 22 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   PROVINCIA        1620160 non-null  object        
 1   MARCA            1620160 non-null  object        
 2   MODELO           1620160 non-null  object        
 3   PROVINCIA_MATR   1620160 non-null  object        
 4   FECHA_MATR       1620160 non-null  datetime64[ns]
 5   FECHA_PRIM_MATR  1620160 non-null  datetime64[ns]
 6   TIPO_DGT         1620160 non-null  object        
 7   CAT_EURO         1620160 non-null  object        
 8   RENTING          1620160 non-null  object        
 9   TARA             1620160 non-null  int64         
 10  PESO_MAX         1620160 non-null  int64         
 11  CILINDRADA       1620160 non-null  int64         
 12  POTENCIA         1620160 non-null  float32       
 13  KW               1620160 non-null  float32       
 14  PROPU

In [38]:
df.dtypes

PROVINCIA                  object
MARCA                      object
MODELO                     object
PROVINCIA_MATR             object
FECHA_MATR         datetime64[ns]
FECHA_PRIM_MATR    datetime64[ns]
TIPO_DGT                   object
CAT_EURO                   object
RENTING                    object
TARA                        int64
PESO_MAX                    int64
CILINDRADA                  int64
POTENCIA                  float32
KW                        float32
PROPULSION                 object
CATELECT                   object
CONSUMO                   float64
AUTONOMIA                 float64
ALIMENTACION               object
TIPO_DISTINTIVO            object
EMISIONES_EURO             object
EMISIONES_CO2             float64
dtype: object

In [39]:
df.head()

,PROVINCIA,MARCA,MODELO,PROVINCIA_MATR,FECHA_MATR,FECHA_PRIM_MATR,TIPO_DGT,CAT_EURO,RENTING,TARA,...,POTENCIA,KW,PROPULSION,CATELECT,CONSUMO,AUTONOMIA,ALIMENTACION,TIPO_DISTINTIVO,EMISIONES_EURO,EMISIONES_CO2
ID,,,,,,,,,,,,,,,,,,,,,
13,Huelva,RENAULT,ARKANA E-TECH,Segovia,2022-06-28,2022-06-28,TURISMOS,M1,N,1435,...,11.64,69.000000,Gasolina,(HEV) Eléctrico Híbrido,0.0,0.0,Monofuel,ECO,EURO 6AP,108.0
22,Girona,HYUNDAI,KONA,Granada,2022-03-07,2022-03-07,TURISMOS,M1,N,1376,...,11.56,77.199997,Gasolina,(HEV) Eléctrico Híbrido,0.0,0.0,Monofuel,ECO,EURO 6AP,112.0
36,Alicante,KIA,NIRO,Álava,2018-03-28,2018-03-28,TURISMOS,M1,N,1425,...,11.56,77.199997,Gasolina,(HEV) Eléctrico Híbrido,0.0,0.0,Monofuel,ECO,EURO 6W,88.0
106,Ciudad Real,LEXUS,LEXUS NX300H,Madrid,2016-03-29,2016-03-29,TURISMOS,M1,N,1815,...,0.15,114.000000,Gasolina,(HEV) Eléctrico Híbrido,0.0,0.0,Monofuel,ECO,EURO 6W,121.0
143,Alicante,FORD,PUMA,La Coruña,2021-12-16,2021-12-16,TURISMOS,M1,N,1238,...,7.82,91.900002,Gasolina,(HEV) Eléctrico Híbrido,0.0,0.0,Monofuel,ECO,EURO 6AP,129.0


In [40]:
print(df.columns[df.isna().any()].tolist())
print(df.isnull().sum())


[]
PROVINCIA          0
MARCA              0
MODELO             0
PROVINCIA_MATR     0
FECHA_MATR         0
FECHA_PRIM_MATR    0
TIPO_DGT           0
CAT_EURO           0
RENTING            0
TARA               0
PESO_MAX           0
CILINDRADA         0
POTENCIA           0
KW                 0
PROPULSION         0
CATELECT           0
CONSUMO            0
AUTONOMIA          0
ALIMENTACION       0
TIPO_DISTINTIVO    0
EMISIONES_EURO     0
EMISIONES_CO2      0
dtype: int64
